<a href="https://colab.research.google.com/github/aaayup/anomaly_detection/blob/main/code/ARIMA(normal_all_predict).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data 불러오기

In [1]:
#importing important libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA

In [2]:
df=pd.read_csv('../../simulation/re/데이터_젊음_중년_장년/젊음_격렬_10.csv')
df = df.copy()
df.head()

,update_time,heart_rate1,event1,heart_rate2,event2,heart_rate3,event3,heart_rate4,event4,heart_rate5,...,heart_rate996,event996,heart_rate997,event997,heart_rate998,event998,heart_rate999,event999,heart_rate1000,event1000
0,2023-10-01 00:00:00,105,0,192,1,120,0,121,0,175,...,138,0,137,0,84,0,90,0,141,0
1,2023-10-01 00:10:00,97,0,100,0,185,1,165,0,173,...,160,0,135,0,152,0,155,0,151,0
2,2023-10-01 00:20:00,164,0,158,0,188,1,146,0,132,...,150,0,154,0,158,0,124,0,150,0
3,2023-10-01 00:30:00,142,0,113,0,148,0,132,0,123,...,145,0,118,0,136,0,86,0,160,0
4,2023-10-01 00:40:00,146,0,116,0,94,0,138,0,196,...,105,0,142,0,132,0,146,0,111,0


#반복문

In [3]:
df_copy = df.copy()

# 데이터를 df_train과 df_test로 분할
df_train = df_copy[:1008]
df_test = df_copy[1008:]

print(df_copy.head())

print(df_train.shape)
print(df_test.shape)


           update_time  heart_rate1  event1  heart_rate2  event2  heart_rate3  \
0  2023-10-01 00:00:00          105       0          192       1          120   
1  2023-10-01 00:10:00           97       0          100       0          185   
2  2023-10-01 00:20:00          164       0          158       0          188   
3  2023-10-01 00:30:00          142       0          113       0          148   
4  2023-10-01 00:40:00          146       0          116       0           94   

   event3  heart_rate4  event4  heart_rate5  ...  heart_rate996  event996  \
0       0          121       0          175  ...            138         0   
1       1          165       0          173  ...            160         0   
2       1          146       0          132  ...            150         0   
3       0          132       0          123  ...            145         0   
4       0          138       0          196  ...            105         0   

   heart_rate997  event997  heart_rate998  event99

In [4]:
for i in range(1, 1001):
    # ARIMA 모델 학습
    model = ARIMA(df_train[f'heart_rate{i}'], order=(2, 0, 1))
    result = model.fit()

    start_idx = 1008  # 테스트 데이터셋의 시작 인덱스
    end_idx = 1008 + len(df_test)  # 테스트 데이터셋의 끝 인덱스

    forecast = result.get_prediction(start=start_idx, end=end_idx, dynamic=False)

    # 신뢰구간과 예측값 할당
    forecast_ci = forecast.conf_int()

  # 데이터프레임에 신뢰구간 추가
    df_test[f'lower_{i}'] = forecast_ci.iloc[:, 0]
    df_test[f'upper_{i}'] = forecast_ci.iloc[:, 1]

    # 'pred_y{i}' 열이 이미 존재하는 경우 삭제하고 다시 추가 (insert 메서드 사용)
    if f'pred_y{i}' in df_test.columns:
        df_test.drop(columns=[f'pred_y{i}'], inplace=True)

    # 'pred_y{i}' 열을 'event{i}' 뒤에 생성
    df_test.insert(df_test.columns.get_loc(f'event{i}') + 1, f'pred_y{i}', np.where((df_test[f'heart_rate{i}'] < df_test[f'lower_{i}']) | (df_test[f'heart_rate{i}'] > df_test[f'upper_{i}']), 1, 0))

    # 'pred{i}', 'lower_{i}', 'upper_{i} 삭제
    if f'pred{i}' in df_test.columns:
        df_test.drop(columns=[f'pred{i}'], inplace = True)
    if f'lower_{i}' in df_test.columns:
        df_test.drop(columns=[f'lower_{i}'], inplace = True)
    if f'upper_{i}' in df_test.columns:
        df_test.drop(columns=[f'upper_{i}'], inplace = True)

c:\Users\jun99\OneDrive - 계명대학교\바탕 화면\Project\TP\분석 연습\ARIMA\arimafd\arima\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\jun99\OneDrive - 계명대학교\바탕 화면\Project\TP\분석 연습\ARIMA\arimafd\arima\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\jun99\AppData\Local\Temp\ipykernel_5228\1820725872.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[f'lower_{i}'] = forecast_ci.iloc[:, 0]
C:\Users\ju

In [5]:
df_test

,update_time,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
1008,2023-10-08 00:00:00,109,0,0,147,0,0,119,0,0,...,0,131,0,0,116,0,0,110,0,0
1009,2023-10-08 00:10:00,140,0,0,136,0,0,103,0,0,...,0,125,0,0,161,0,0,155,0,0
1010,2023-10-08 00:20:00,159,0,0,76,1,0,102,0,0,...,0,80,0,0,88,0,0,112,0,0
1011,2023-10-08 00:30:00,132,0,0,89,0,0,106,0,0,...,0,118,0,0,145,0,0,118,0,0
1012,2023-10-08 00:40:00,178,0,0,102,0,0,110,0,0,...,0,148,0,0,150,0,0,115,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,2023-10-10 08:10:00,138,0,0,173,0,0,122,0,0,...,1,139,0,0,149,0,0,153,0,0
1346,2023-10-10 08:20:00,111,0,0,126,0,0,151,0,0,...,0,161,0,0,115,0,0,102,0,0
1347,2023-10-10 08:30:00,138,0,0,155,0,0,188,1,1,...,0,88,0,0,156,0,0,141,0,0
1348,2023-10-10 08:40:00,216,1,1,134,0,0,135,0,0,...,0,104,0,0,101,0,0,87,0,0


In [6]:
from performance import performance

In [7]:
result = pd.DataFrame()

for i in range(1,1001):

    TP = len(df_test[(df_test[f'event{i}']==1)&(df_test[f'pred_y{i}']==1)])
    TN = len(df_test[(df_test[f'event{i}']==0)&(df_test[f'pred_y{i}']==0)])
    FN = len(df_test[(df_test[f'event{i}']==1)&(df_test[f'pred_y{i}']==0)])
    FP = len(df_test[(df_test[f'event{i}']==0)&(df_test[f'pred_y{i}']==1)])


    result_i = performance(TP=TP, TN=TN, FP=FP, FN=FN, BETA=1)
    
    result = pd.concat([result, result_i])


result.head(2)

,Sensitivity,Specificity,Accuracy,PPV,NPV,F_1 score
0,59.375,100.0,96.198830,100.0,95.975232,74.509804
0,62.500,100.0,96.491228,100.0,96.273292,76.923077


In [8]:
정규_격렬_젊음_10 = pd.DataFrame(index=['mean'])
정규_격렬_젊음_10['mean_sens'] = result['Sensitivity'].mean()정규_격렬_젊음_10['std_sens'] = result['Sensitivity'].std()정규_격렬_젊음_10['mean_spec'] = result['Specificity'].mean()정규_격렬_젊음_10['std_spec'] = result['Specificity'].std()정규_격렬_젊음_10['mean_accu'] = result['Accuracy'].mean()정규_격렬_젊음_10['std_accu'] = result['Accuracy'].std()정규_격렬_젊음_10['mean_ppv'] = result['PPV'].mean()정규_격렬_젊음_10['std_ppv'] = result['PPV'].std()정규_격렬_젊음_10['mean_npv'] = result['NPV'].mean()정규_격렬_젊음_10['std_npv'] = result['NPV'].std()정규_격렬_젊음_10['mean_F1_score'] = result['F_1 score'].mean()정규_격렬_젊음_10['std_F1_score'] = result['F_1 score'].std()
정규_격렬_젊음_10

,mean_sens,std_sens,mean_spec,std_spec,mean_accu,std_accu,mean_ppv,std_ppv,mean_npv,std_npv,mean_F1_score,std_F1_score
mean,57.566925,11.188179,100.0,0.0,96.277778,1.173284,100.0,0.0,96.083508,1.222341,72.408935,9.371538


In [9]:
정규_격렬_젊음_10.to_csv('../../../분석 연습/result/정규_격렬_젊음_10.csv', encoding='cp949')